In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import PIL
#Tensorflow libs
import tensorflow as tf
import os

In [ ]:
train_data_path = './Fruit-262'
test_data_path = './Fruit-262/'
validate_data_path = './Fruit-262/'
os.path.exists(train_data_path)
os.path.exists(test_data_path)
os.path.exists(validate_data_path)

print(tf.config.list_physical_devices('GPU'))

In [ ]:
img_width=100
img_height=100

In [ ]:
data_train = tf.keras.utils.image_dataset_from_directory(
    train_data_path,
    validation_split=0.8,
    subset="training",
    seed=123,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32
)

data_test = tf.keras.utils.image_dataset_from_directory(
    train_data_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    shuffle=True,
    image_size=(img_width, img_height),
    batch_size=32
)

data_val= tf.keras.utils.image_dataset_from_directory(
    validate_data_path,
    shuffle=True,
    image_size=(img_width,img_height),
    batch_size=32,
)


In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                             shear_range=0.4,
                             zoom_range=0.2,
                             rotation_range=50,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                             shear_range=0.4,
                             zoom_range=0.2,
                             rotation_range=50,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest')

train_data = train_generator.flow_from_directory(
    train_data_path,
    target_size=(img_width,img_height),
    batch_size=100,
    class_mode='categorical'
)

test_data = test_generator.flow_from_directory(
    test_data_path,
    target_size=(img_width,img_height),
    batch_size=100,
    class_mode='categorical'
)

validation_data = test_generator.flow_from_directory(
    train_data_path,
    target_size=(img_width,img_height),
    batch_size=100,
    class_mode='categorical'
)



In [ ]:
data_cat=data_train.class_names
data_cat
len(data_cat)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in data_train.take(1):
    for i in range(min(9, images.shape[0])):
        plt.subplot(3, 3, i + 1)
        # Normalize the image pixels to [0, 1] range
        plt.imshow(images[i].numpy() / 255.0)
        plt.title(data_cat[labels[i].numpy()])
        plt.axis("off")
plt.show()

In [ ]:
#create augument sequences
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",
                      input_shape=(img_width,img_height,3)),
    tf.keras.layers.RandomRotation(0.3),
    tf.keras.layers.RandomZoom(0.3),
  ]
)


In [ ]:
#augumented data
plt.figure(figsize=(10, 10))
for images, _ in data_train.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_images[0].numpy() / 255.0)
        plt.axis('off')
plt.show()


In [ ]:
# Load existing model
# If model is not available, create a new model
if os.path.exists('fruits_classification.keras'):
    model = tf.keras.models.load_model('fruits_classification.keras')
else:
    model = tf.keras.models.Sequential([
        data_augmentation,
        # first convolution
        tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(img_width, img_height, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        # second convolution
        tf.keras.layers.Conv2D(64, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        # third convolution layer
        tf.keras.layers.Conv2D(128, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        # fourth convolution layer
        tf.keras.layers.Conv2D(256, (3,3), activation="relu"),
        tf.keras.layers.MaxPooling2D(2,2),
        # flatten before feeding into Dense neural network. 
        tf.keras.layers.Flatten(),
        # 512 neurons in the hidden layer
        tf.keras.layers.Dense(128, activation="relu"),
        # 141 = 141 different categories
        # softmax takes a set of values and effectively picks the biggest one. for example if the output layer has
        # [0.1,0.1,0.5,0.2,0.1], it will take it and turn it into [0,0,1,0,0]
        tf.keras.layers.Dense(len(data_cat), activation="softmax")
    ])

model.compile(optimizer='adam',
                  loss=tf.keras.losses.CategoricalCrossentropy(),
                  metrics=['accuracy'])

epochs = 10
history = model.fit(
   train_data,
    epochs = epochs,  # Increase the number of epochs for better training
    validation_data=validation_data,
    validation_steps=100,
    steps_per_epoch=100,
)
model.save('fruits_classification.keras')

In [ ]:
# Extracting loss and accuracy from history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plotting loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
fruit_classifier = tf.keras.models.load_model('fruits_classification.keras')
test_loss, test_accuracy = fruit_classifier.evaluate(test_data)
print("Test Accuracy:", test_accuracy)

In [ ]:
def predict_image(img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create batch axis

    predictions = fruit_classifier.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    return data_cat[np.argmax(score)]

In [ ]:
import random

# Get all subfolders in the test data path
subfolders = [f.path for f in os.scandir(test_data_path) if f.is_dir()]

# Limit the number of subplots to 100
for i, subfolder in enumerate(subfolders[:100]):
    if i % 9 == 0:
        if i != 0:
            plt.tight_layout()
            plt.show()
        plt.figure(figsize=(10, 10))
    
    # Get all files in the subfolder
    files = os.listdir(subfolder)
    # Select a random file
    random_file = random.choice(files)
    # Load the image
    img_path = os.path.join(subfolder, random_file)
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_width, img_height))
    
    plt.subplot(3, 3, (i % 9) + 1)
    plt.imshow(img)
    plt.title('actual : '+os.path.basename(subfolder) + ' \npredicted : ' + predict_image(img))
    plt.axis('off')

    # Adjust space between images
    plt.subplots_adjust(hspace=0.5, wspace=0.5)

plt.tight_layout()
plt.show()

In [ ]:
image = tf.keras.preprocessing.image.load_img('./watermelon.jpg', target_size=(img_width, img_height))
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Image")
plt.axis('off')

predicted_label = predict_image(image)

plt.subplot(1, 2, 2)
plt.imshow(image)
plt.title(f"Predicted: {predicted_label}")
plt.axis('off')
